In [ ]:
from ultralytics import YOLO

# Explanation of the parameters:


- | batch      | Number of images processed in parallel per step. A larger batch can stabilize training but uses more RAM.     |

- | workers    | How many subprocesses load and preprocess data. More workers can speed up data IO but also consume more memory. |
- | cache      | If True, loads all images into RAM/disk cache on first epoch, speeding up subsequent epochs at the cost of initial load time and memory. |
- | half       | Enables automatic mixed precision (FP16) during training. Cuts memory usage (≈½) and often speeds up training with minimal impact on accuracy. |
- | accumulate | Gradient accumulation steps:
        After every batch × accumulate forward passes, gradients are summed and a single optimizer step is taken.
        This effectively gives you an “effective batch size” of batch * accumulate (here 4×2=8) without needing the RAM for 8 images at once. |
- | lr0        | Initial learning rate for the optimizer. Too high→unstable; too low→slow convergence.                           |
- | momentum   | Momentum factor in SGD: smooths updates by combining current gradient with a fraction of the previous update.   |
- | weight_decay | L2 regularization coefficient: penalizes large weights to reduce overfitting, especially important with small batches. |
- | patience   | Number of epochs with no improvement in the monitored metric (mAP) before early stopping kicks in.      |

In [11]:

model = YOLO("yolo11s.pt")

results = model.train(
    data="YOLO_format/data.yaml",
    device="mps",
    epochs=5,
    imgsz=512,
    batch=4,
    workers=2,
    cache=True,
    half=True,
    # accumulate=2,
    lr0=0.01,
    momentum=0.9,
    weight_decay=0.0005,
    name="affectnet_m2_8gb"
)

Ultralytics 8.3.130 🚀 Python-3.12.9 torch-2.6.0 MPS (Apple M2)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=YOLO_format/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.9, mosaic=1.0, multi_scale=False, name=affectnet_m2_8gb6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, 

train: Scanning /Users/nt/downloads/ml_3year_final/YOLO_format/train/labels... 17101 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17101/17101 [00:02<00:00, 6258.15it/s]


train: New cache created: /Users/nt/downloads/ml_3year_final/YOLO_format/train/labels.cache
WARNING ⚠️ train: 18.8GB RAM required to cache images with 50% safety margin but only 1.4/8.0GB available, not caching images
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 40.4±25.9 MB/s, size: 9.1 KB)


val: Scanning /Users/nt/downloads/ml_3year_final/YOLO_format/valid/labels... 5406 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5406/5406 [00:00<00:00, 6288.72it/s]


val: New cache created: /Users/nt/downloads/ml_3year_final/YOLO_format/valid/labels.cache
WARNING ⚠️ val: 5.9GB RAM required to cache images with 50% safety margin but only 1.4/8.0GB available, not caching images
Plotting labels to runs/detect/affectnet_m2_8gb6/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.9' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 512 train, 512 val
Using 0 dataloader workers
Logging results to runs/detect/affectnet_m2_8gb6
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/5      3.65G     0.2566      2.565      1.035         10        512:  15%|█▍        | 622/4276 [08:24<49:24,  1.23it/s]  


KeyboardInterrupt: 

In [ ]:
metrics = model.val(data="YOLO_format/data.yaml", imgsz=640, batch=8)
print(metrics)   # prints mAP, precision, recall, etc.

In [ ]:
# run detection on your test images
preds = model.predict(
    source="YOLO_format/test/images",
    conf=0.5,
    save=True   # saves into runs/predict
)

In [ ]:
from PIL import Image
img = Image.open('runs/predict/image1.jpg')
display(img)

In [ ]:
import glob, random, cv2, matplotlib.pyplot as plt

# pick a random train image
img_path = random.choice(glob.glob('YOLO_format/train/images/*.jpg'))
lbl_path = img_path.replace('/images/', '/labels/').replace('.jpg', '.txt')

img = cv2.imread(img_path)
h, w = img.shape[:2]
for line in open(lbl_path):
    c, x, y, bw, bh = map(float, line.split())
    x1 = int((x - bw/2) * w)
    y1 = int((y - bh/2) * h)
    x2 = int((x + bw/2) * w)
    y2 = int((y + bh/2) * h)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')